In [2]:
import pandas as pd
import numpy as np

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('static/raw-data/institution_details.csv')

# Filter the DataFrame to include only rows where "Graduate offering (HD2021)" is 1
df = df[df['Graduate offering (HD2021)'] == 1]

# Save the cleaned dataset to a new CSV file
df.to_csv('static/clean_dataset/graduate_school_records.csv', index=False)

In [3]:
# Load the cleaned graduate school dataset
grad_school_df = pd.read_csv('static/clean_dataset/graduate_school_records.csv')

# Load the admission considerations dataset
admission_considerations_df = pd.read_csv('static/raw-data/admission_considerations.csv')

# Filter the admission considerations dataset to only include rows where "UnitID" matches a value in the cleaned graduate school dataset
filtered_admission_considerations_df = admission_considerations_df[admission_considerations_df['UnitID'].isin(grad_school_df['UnitID'])]

# Load the enrollment details dataset
enrollment_details_df = pd.read_csv('static/raw-data/enrollment_details.csv')

# Filter the enrollment details dataset to only include rows where "UnitID" matches a value in the cleaned graduate school dataset
filtered_enrollment_details_df = enrollment_details_df[enrollment_details_df['UnitID'].isin(grad_school_df['UnitID'])]

# Load the ethnicity details dataset
ethnicity_details_df = pd.read_csv('static/raw-data/ethnicity_details.csv')

# Filter the ethnicity details dataset to only include rows where "UnitID" matches a value in the cleaned graduate school dataset
filtered_ethnicity_details_df = ethnicity_details_df[ethnicity_details_df['UnitID'].isin(grad_school_df['UnitID'])]

# Load the tuition details dataset
tuition_details_df = pd.read_csv('static/raw-data/tuition_details.csv')

# Filter the tuition details dataset to only include rows where "UnitID" matches a value in the cleaned graduate school dataset
filtered_tuition_details_df = tuition_details_df[tuition_details_df['UnitID'].isin(grad_school_df['UnitID'])]

# Save the filtered datasets to new CSV files
filtered_admission_considerations_df.to_csv('static/clean_dataset/filtered_admission_considerations.csv', index=False)
filtered_enrollment_details_df.to_csv('static/clean_dataset/filtered_enrollment_details.csv', index=False)
filtered_ethnicity_details_df.to_csv('static/clean_dataset/filtered_ethnicity_details.csv', index=False)
filtered_tuition_details_df.to_csv('static/clean_dataset/filtered_tuition_details.csv', index=False)


In [4]:
# read the filtered data from admission_considerations.csv
admission_df = pd.read_csv("static/clean_dataset/filtered_admission_considerations.csv")

# read the filtered data from enrollment_details.csv
enrollment_df = pd.read_csv("static/clean_dataset/filtered_enrollment_details.csv")

# read the filtered data from ethnicity_details.csv
ethnicity_df = pd.read_csv("static/clean_dataset/filtered_ethnicity_details.csv")

# read the filtered data from tuition_details.csv
tuition_df = pd.read_csv("static/clean_dataset/filtered_tuition_details.csv")

# read the filtered data from graduate_school_records.csv
grad_school_df = pd.read_csv("static/clean_dataset/graduate_school_records.csv")

# merge the data from all the dataframes on the 'UnitID' column
merged_df = pd.merge(admission_df, enrollment_df, on='UnitID')
merged_df = pd.merge(merged_df, ethnicity_df, on='UnitID')
merged_df = pd.merge(merged_df, tuition_df, on='UnitID', suffixes=('_tuition', '_merged'))
merged_df = pd.merge(merged_df, grad_school_df, on='UnitID')

# write the merged data to a new csv file
merged_df.to_csv("static/clean_dataset/combined_data.csv", index=False)


In [5]:
df = pd.read_csv('static/clean_dataset/combined_data.csv')

In [6]:
df.isna().sum()

UnitID                                  0
Institution Name_x                      0
Open admission policy (IC2021)         21
Secondary school GPA (ADM2021)        725
Secondary school rank (ADM2021)       725
                                     ... 
ZIP code (HD2021)                       0
Institutional category (HD2021)         0
Highest level of offering (HD2021)      0
Undergraduate offering (HD2021)         0
Graduate offering (HD2021)              0
Length: 366, dtype: int64

In [7]:
df.isna().sum().sum()

188502

In [8]:
import pandas as pd
import numpy as np

# Load the data
admissions_df = pd.read_csv('static/clean_dataset/filtered_admission_considerations.csv')

# Calculate the percentage of missing values in each column
column_missing_perc = admissions_df.isnull().sum() / len(admissions_df)

# Calculate the means of columns containing numeric values
numeric_columns = admissions_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns.remove('UnitID') # Exclude the 'UnitID' column
column_means = admissions_df[numeric_columns].mean()

# Fill missing values in columns containing less than or equal to 40% missing values
for column_name in column_missing_perc.index:
    if column_missing_perc[column_name] <= 0.4 and column_name in numeric_columns:
        column_mean = column_means[column_name]
        admissions_df[column_name].fillna(column_mean, inplace=True)

# Write the cleaned data to a new file
admissions_df.to_csv('static/clean_dataset/admission_considerations_filtered_cleaned.csv', index=False)


In [9]:
df = pd.read_csv('static/clean_dataset/admission_considerations_filtered_cleaned.csv')

In [10]:
df.isna().sum()

UnitID                                                          0
Institution Name                                                0
Open admission policy (IC2021)                                  0
Secondary school GPA (ADM2021)                                  0
Secondary school rank (ADM2021)                                 0
Secondary school record (ADM2021)                               0
Completion of college-preparatory program (ADM2021)             0
Recommendations (ADM2021)                                       0
Formal demonstration of competencies (ADM2021)                  0
Admission test scores (ADM2021)                                 0
Other Test (Wonderlic  WISC-III  etc.) (ADM2021)                0
TOEFL (Test of English as a Foreign Language (ADM2021)          0
Open admission policy (IC2020)                                  0
Secondary school GPA (ADM2020_RV)                               0
Secondary school rank (ADM2020_RV)                              0
Secondary 

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

def fill_missing_values(file_path):
    df = pd.read_csv(file_path)

    # Filter Alabama colleges
    alabama_colleges = df[df['state'] == 'AL']

    # Helper function to calculate mean with standard deviation of 2
    def mean_within_2_std(column):
        mean = column.mean()
        std = column.std()
        return mean - (2 * std), mean + (2 * std)

    # Helper function to calculate missing values based on the conditions
    def calculate_missing_values(row):
        men = row['applicants_men_2021']
        women = row['applicants_women_2021']
        total = row['applicants_total_2021']

        if pd.isnull(men) and pd.isnull(women) and pd.isnull(total):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            women_min, women_max = mean_within_2_std(alabama_colleges['applicants_women'].dropna())
            men_mean = (men_min + men_max) / 2
            women_mean = (women_min + women_max) / 2
            row['applicants_menstatic/'] = men_mean
            row['applicants_women'] = women_mean
            row['applicants_total_2021'] = men_mean + women_mean

        elif pd.isnull(men) and pd.isnull(women):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            men_mean = (men_min + men_max) / 2
            row['applicants_men_2021'] = men_mean
            row['applicants_women_2021'] = total - men_mean

        elif pd.isnull(total):
            total_min, total_max = mean_within_2_std(alabama_colleges['applicants_total_2021'].dropna())
            total_mean = (total_min + total_max) / 2
            if pd.isnull(men):
                row['applicants_men_2021'] = total_mean - women
            else:
                row['applicants_women_2021'] = total_mean - men
            row['applicants_total_2021'] = total_mean

        elif pd.isnull(men):
            row['applicants_men_2021'] = total - women

        elif pd.isnull(women):
            row['applicants_women_2021'] = total - men

        return row

    # Apply the helper function to fill missing values
    df = df.apply(calculate_missing_values, axis=1)

    # Save the filled DataFrame to a new CSV file
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    df.to_csv(f"filled_college_data_{current_time}.csv", index=False)

# Usage example
file_path = "C:\\College_Compass_backend\\college-compass-backend-flask\\static\\with_missing_values\\combined_with_lat_long.csv"
fill_missing_values(file_path)

In [6]:
import pandas as pd
import numpy as np

def fill_missing_values(df):
    for index, row in df.iterrows():
        # Check if one of the three columns is missing
        if row.isnull().sum() == 1:
            if pd.isnull(row['applicants_total_2021']):
                df.at[index, 'applicants_total_2021'] = row['applicants_men_2021'] + row['applicants_women_2021']
            elif pd.isnull(row['applicants_men_2021']):
                df.at[index, 'applicants_men_2021'] = row['applicants_total_2021'] - row['applicants_women_2021']
            else:
                df.at[index, 'applicants_women_2021'] = row['applicants_total_2021'] - row['applicants_men_2021']
        
        # Check if two of the three columns are missing
        elif row.isnull().sum() == 2:
            alabama_df = df[df['state'] == 'AL']
            
            if pd.isnull(row['applicants_total_2021']):
                mean_men = alabama_df['applicants_men_2021'].mean()
                mean_women = alabama_df['applicants_women_2021'].mean()
                total_applicants = mean_men + mean_women
                
                if pd.isnull(row['applicants_men_2021']):
                    df.at[index, 'applicants_total_2021'] = total_applicants
                    df.at[index, 'applicants_men_2021'] = mean_men
                else:
                    df.at[index, 'applicants_total_2021'] = total_applicants
                    df.at[index, 'applicants_women_2021'] = mean_women
                    
            else:
                mean_total = alabama_df['applicants_total_2021'].mean()
                if pd.isnull(row['applicants_men_2021']):
                    df.at[index, 'applicants_men_2021'] = mean_total - row['applicants_women_2021']
                else:
                    df.at[index, 'applicants_women_2021'] = mean_total - row['applicants_men_2021']
                
        # Check if all three columns are missing
        elif row.isnull().sum() == 3:
            alabama_df = df[df['state'] == 'AL']
            mean_men = alabama_df['applicants_men_2021'].mean()
            mean_women = alabama_df['applicants_women_2021'].mean()
            
            df.at[index, 'applicants_men_2021'] = mean_men
            df.at[index, 'applicants_women_2021'] = mean_women
            df.at[index, 'applicants_total_2021'] = mean_men + mean_women

    return df


csv_path = r"C:\\College_Compass_backend\\college-compass-backend-flask\\static\\with_missing_values\\combined_with_lat_long.csv"
df = pd.read_csv(csv_path)

filled_df = fill_missing_values(df)
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filled_df.to_csv(f"filled_college_data_{current_time}.csv", index=False)

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

def fill_missing_values(file_path):
    df = pd.read_csv(file_path)

    # Filter Alabama colleges
    def filter_alabama_colleges(df):
        return df[df['state'] == 'AL']

    alabama_colleges = filter_alabama_colleges(df)

    # Helper function to calculate mean with standard deviation of 2
    def mean_within_2_std(column):
        mean = column.mean()
        std = column.std()
        return mean - (std), mean + (std)

    # Helper function to calculate missing values based on the conditions
    def calculate_missing_values(row):
        men = row['applicants_men_2021']
        women = row['applicants_women_2021']
        total = row['applicants_total_2021']

        if pd.isnull(men) and pd.isnull(women) and pd.isnull(total):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            women_min, women_max = mean_within_2_std(alabama_colleges['applicants_women_2021'].dropna())
            men_mean = randint(men_min, men_max)
            women_mean = randint(women_min, women_max)
            row['applicants_men_2021'] = men_mean
            row['applicants_women_2021'] = women_mean
            row['applicants_total_2021'] = men_mean + women_mean

        elif pd.isnull(men) and pd.isnull(women):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            men_mean = randint(men_min, men_max)
            row['applicants_men_2021'] = men_mean
            row['applicants_women_2021'] = total - men_mean

        elif pd.isnull(total):
            total_min, total_max = mean_within_2_std(alabama_colleges['applicants_total_2021'].dropna())
            total_mean = randint(total_min, total_max)
            if pd.isnull(men):
                row['applicants_men_2021'] = total_mean - women
            else:
                row['applicants_women_2021'] = total_mean - men
            row['applicants_total_2021'] = total_mean

        elif pd.isnull(men):
            row['applicants_men_2021'] = total - women

        elif pd.isnull(women):
            row['applicants_women_2021'] = total - men

        return row

    # Apply the helper function to fill missing values
    df = df.apply(calculate_missing_values, axis=1)

    # Update Alabama colleges DataFrame
    alabama_colleges = filter_alabama_colleges(df)

    # Save the filled DataFrame to a new CSV file
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    df.to_csv(f"filled_college_data_{current_time}.csv", index=False)

# Usage example
file_path = "C:\\College_Compass_backend\\college-compass-backend-flask\\static\\with_missing_values\\combined_with_lat_long.csv"
fill_missing_values(file_path)

NameError: name 'randint' is not defined

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime

def fill_missing_values(file_path):
    df = pd.read_csv(file_path)

    # Filter Alabama colleges
    def filter_alabama_colleges(df):
        return df[df['state'] == 'AL']

    # Helper function to calculate mean with standard deviation of 2
    def mean_within_2_std(column):
        mean = column.mean()
        std = column.std()
        return mean - (2 * std), mean + (2 * std)

    # Helper function to calculate missing values based on the conditions
    def calculate_missing_values(row, alabama_colleges):
        men = row['applicants_men_2021']
        women = row['applicants_women_2021']
        total = row['applicants_total_2021']

        if pd.isnull(men) and pd.isnull(women) and pd.isnull(total):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            women_min, women_max = mean_within_2_std(alabama_colleges['applicants_women_2021'].dropna())
            men_mean = (men_min + men_max) / 2
            women_mean = (women_min + women_max) / 2
            row['applicants_men_2021'] = men_mean
            row['applicants_women_2021'] = women_mean
            row['applicants_total_2021'] = men_mean + women_mean

        elif pd.isnull(men) and pd.isnull(women):
            men_min, men_max = mean_within_2_std(alabama_colleges['applicants_men_2021'].dropna())
            men_mean = (men_min + men_max) / 2
            row['applicants_men_2021'] = men_mean
            row['applicants_women_2021'] = total - men_mean

        elif pd.isnull(total):
            total_min, total_max = mean_within_2_std(alabama_colleges['applicants_total_2021'].dropna())
            total_mean = (total_min + total_max) / 2
            if pd.isnull(men):
                row['applicants_men_2021'] = total_mean - women
            else:
                row['applicants_women_2021'] = total_mean - men
            row['applicants_total_2021'] = total_mean

        elif pd.isnull(men):
            row['applicants_men_2021'] = total - women

        elif pd.isnull(women):
            row['applicants_women_2021'] = total - men

        return row

    for index, row in df.iterrows():
        # Apply the helper function to fill missing values
        df.at[index] = calculate_missing_values(row, filter_alabama_colleges(df))

    # Save the filled DataFrame to a new CSV file
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    df.to_csv(f"filled_college_data_{current_time}.csv", index=False)

# Usage example
file_path = "C:\\College_Compass_backend\\college-compass-backend-flask\\static\\with_missing_values\\combined_with_lat_long.csv"
fill_missing_values(file_path)


InvalidIndexError: You can only assign a scalar value not a <class 'pandas.core.series.Series'>

In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
import random
import math

# Filter US colleges based on state
state_abbreviations = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
    'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
}

years = ["_2021", "_2020", "_2019", "_2018", "_2017"]

columns = [
    ["applicants_men", "applicants_women", "applicants_total"],
    ["admissions_men", "admission_women", "admissions_total"],
    ["enrolled_ft_men_total", "enrolled_ft_women_total", "enrolled_ft_total"],
    ["enrolled_pt_men_total", "enrolled_pt_women_total", "enrolled_pt_total"],
    ["grand_total_men", "grand_total_women", "grand_total"],
    ["am_ind_alaska_total_men", "am_ind_alaska_total_women", "am_ind_alaska_total"],
    ["asian_men", "asian_women", "asian_total"],
    ["bl_af_men", "bl_af_women", "bl_af_total"],
    ["hisp_men", "hisp_women", "hisp_total"],
    ["haw_pac_men", "haw_pac_women", "haw_pac_total"],
    ["white_men", "white_women", "white_total"],
    ["two_plus_race_men", "two_plus_race_women", "two_plus_race_total"],
    ["unk_men", "unk_women", "unk_total"],
    ["alien_men", "alien_women", "alien_total"]
]
    
# column_men = 'hisp_men_2018'  
# column_women = 'hisp_women_2018'
# column_total = 'hisp_total_2018'

def filter_us_colleges(df, state):
    return df[df['state'] == state]

# Helper function to calculate mean with standard deviation of 2
def mean_within_2_std(column):
    mean = column.mean()
    std = column.std()
    # while True:
    #     try:
            
    #     except OverflowError:
    #         print(mean, std)
    #         low = math.floor(mean)
    #     if low > 0:
    #         break
    #     n += 1
    low = math.floor(mean - (std/2))
    if low < 0:
        low = math.floor(mean)
        high = math.ceil(mean + (std))
    else:
        high = math.ceil(mean + (std/2))
    return low, high

# Helper function to calculate missing values based on the conditions
def calculate_missing_values(row, state_colleges, column):
    men = row[column[0]]
    women = row[column[1]]
    total = row[column[2]]

    if pd.isnull(men) and pd.isnull(women) and pd.isnull(total):
        men_min, men_max = mean_within_2_std(state_colleges[column[0]].dropna())
        women_min, women_max = mean_within_2_std(state_colleges[column[1]].dropna())
        men_mean = random.randint(men_min, men_max)
        women_mean = random.randint(women_min, women_max)
        row[column[0]] = men_mean
        row[column[1]]= women_mean
        row[column[2]]= men_mean + women_mean

    elif pd.isnull(men) and pd.isnull(women):
        men_min, men_max = mean_within_2_std(state_colleges[column[0]].dropna())
        men_mean = random.randint(men_min, men_max)
        row[column[0]] = men_mean
        row[column[1]]= total - men_mean

    elif pd.isnull(total):
        total_min, total_max = mean_within_2_std(state_colleges[column[2]].dropna())
        total_mean = random.randint(total_min, total_max)
        if pd.isnull(men):
            row[column[0]] = total_mean - women
        else:
            row[column[1]] =total_mean - men
        row[column[2]]= total_mean

    elif pd.isnull(men):
        row[column[0]] = total - women

    elif pd.isnull(women):
        row[column[1]]= total - men

    return row

def fill_missing_values(df, state, column):
    state_colleges = filter_us_colleges(df, state)
    state_colleges = state_colleges.apply(lambda row: calculate_missing_values(row, state_colleges, column), axis=1)
    df.update(state_colleges)
    return df

# Usage example
file_path = "static/with_missing_values/combined_with_lat_long.csv"
df = pd.read_csv(file_path)

for year in years:
    for column in columns:
        mod_columns = [c + year for c in column]
        print(mod_columns)
        for state in state_abbreviations:
            df = fill_missing_values(df, state, mod_columns)

print(df)
#  Save the filled DataFrame to a new CSV file
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# df.to_csv(f"filled_college_data_{current_time}.csv", index=False)
df.to_csv(f"complete_dataset_{current_time}.csv", index=False)


['applicants_men_2021', 'applicants_women_2021', 'applicants_total_2021']
['admissions_men_2021', 'admission_women_2021', 'admissions_total_2021']
['enrolled_ft_men_total_2021', 'enrolled_ft_women_total_2021', 'enrolled_ft_total_2021']
['enrolled_pt_men_total_2021', 'enrolled_pt_women_total_2021', 'enrolled_pt_total_2021']
['grand_total_men_2021', 'grand_total_women_2021', 'grand_total_2021']
['am_ind_alaska_total_men_2021', 'am_ind_alaska_total_women_2021', 'am_ind_alaska_total_2021']
['asian_men_2021', 'asian_women_2021', 'asian_total_2021']
['bl_af_men_2021', 'bl_af_women_2021', 'bl_af_total_2021']
['hisp_men_2021', 'hisp_women_2021', 'hisp_total_2021']
['haw_pac_men_2021', 'haw_pac_women_2021', 'haw_pac_total_2021']
['white_men_2021', 'white_women_2021', 'white_total_2021']
['two_plus_race_men_2021', 'two_plus_race_women_2021', 'two_plus_race_total_2021']
['unk_men_2021', 'unk_women_2021', 'unk_total_2021']
['alien_men_2021', 'alien_women_2021', 'alien_total_2021']
['applicants_men

In [ ]:
import pandas as pd
import numpy as np
import random

def mean_within_2_std(column):
    mean = column.mean()
    std = column.std()
    n = 2
    while True:
        low = np.floor(mean - (std/n)).item()
        if low > 0:
            break
        n += 1
    high = np.ceil(mean + (std/n)).item()
    return low, high

def fill_missing_values(df, columns):
    for column in columns:
        while df[column].isnull().any():
            non_null_column = df[column].dropna()
            for index, value in df[column].iteritems():
                if pd.isnull(value):
                    low, high = mean_within_2_std(non_null_column)
                    filled_value = random.randint(low, high)
                    df.at[index, column] = filled_value
                    non_null_column = df[column].dropna()
    return df

file_path = r"C:\\College_Compass_backend\\college-compass-backend-flask\\static\\with_missing_values\\combined_with_lat_long.csv"
df = pd.read_csv('complete_dataset.csv')

# Replace 'column_name_1', 'column_name_2', ... with the column names you want to fill missing values for
columns_to_fill = [
    'price_in_dist_on_campus_2021',
    'price_in_st_on_campus_2021',
    'price_out_st_on_campus_2021',
    'price_in_dist_off_campus_2021',
    'price_in_st_off_campus_2021',
    'price_out_st_off_campus_2021',
    'price_in_dist_off_campus_family_2021',
    'price_in_st_off_campus_family_2021',
    'price_out_st_off_campus_family_2021',
    'price_other_off_campus_2021',
    'price_room_and_board_off_campus_2021',
    'price_other_off_campus_family_2021',
    'price_room_and_board_off_campus_family_2021'
]

df = fill_missing_values(df, columns_to_fill)

# Save the filled DataFrame to a new CSV file
# output_path = r'C:\College_Compass_backend\college-compass-backend-flask\static\filled_values.csv'
output_path = 'complete_dataset.csv'
df.to_csv(output_path, index=False)
